In [ ]:
"""
Chapter 03: Writing Web Crawler
Date: 18/04/2024
"""

# thu thập thông tin toàn bộ trang web dùng đệ quy

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import datetime
import random
import re


random.seed(datetime.datetime.now())


def getLinks(articleUrl):
    html = urlopen("http://en.wikipedia.org{}".format(articleUrl))
    bs = BeautifulSoup(html.read(), 'html.parser')
    return bs.find('div', {'id': 'bodyContent'}).find_all(
        'a',
        href=re.compile('^(/wiki/)((?!:).*$)')
        )

links = getLinks('/wiki/Kevin_Bacon')
while len(links) > 0:
    newArticle = links[random.randint(0, len(links)-1)].attrs['href']
    print(newArticle)
    links = getLinks(newArticle)

***note***
việc thu thập tất cả các đường link trong một hệ thống web, và tiếp tục truy
vết các đường link con từ những đường link đã lấy được trước đó

=> điều này tạo thành một vòng lặp đệ quy
một hệ thống web mỗi trang có 10 link và có độ sâu là 5 thì đường đương với
5^10 cho tất cả các đường link được tìm thấy. tuy nhiên hiếm thấy trang web nào có thể đạt tới 100.000 đường link.

khi quét một hệ thống web dùng đệ quy, chắc chắn có thể bị lặp lại vô hạn do bị trùng đường link được tìm thấy. điều này có thể giải quyết bằng cách lưu lại và các đường link mới được sưu tầm phải được kiểm tra xem đã tồn tại trước đó hay chưa!

python giới hạn với vòng lặp đệ quy là 1.000 vòng. trừ khi chúng ta phải thiết lập một bộ đếm đệ quy khác cho nó.



In [ ]:
#  một ví dụ đơn giản về chương trình đệ quy cho việc quét toàn bộ web
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

pages = set()

def getLinks(pageUrl):
    html = urlopen('http://en.wikipedia.org{}'.format(pageUrl))
    bs = BeautifulSoup(html, 'html.parser')
    for link in bs.find_add('a', href=re.compile('^(/wiki)')):
        if 'href' in link.attrs:
            if link.attrs['href'] not in pages:
                # we have encountered a new page
                newPage = link.attrs['href']
                print(newPage)
                pages.add(newPage)
                getLinks(newPage)

getLinks('')


# thu thập dữ liệu trên toàn bộ trang web

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

pages = set()

def getLinks(pageUrl):
    global pages
    html = urlopen('http://en.wikipedia.org{}'.format(pageUrl))
    bs = BeautifulSoup(html, 'html.parser')
    try:
        print(bs.h1.get_text())
        print(bs.find(id='mw-content-text').find_all('p')[0])
        print(bs.find(id='ca-edit').find('span')
              .find('a').attrs['href'])
    except AttributeError:
        print('This page is missing something! Contunuing...')
    for link in bs.find_all('a', href=re.compile('^(/wiki/)')):
        if 'href' in link.attrs:
            if link.attrs['href'] not in pages:
                newPage = link.attrs['href']
                print('_'*20)
                print(newPage)
                pages.add(newPage)
                getLinks(newPage)

getLinks('')


***chuyển hướng***

- chuyển hướng máy chủ: URL được thay đổi trước khi tải

- chuyển hướng máy khách: nhìn thấy thông báo "bạn sẽ được chuyển hướng sau xxx giây". thư viện url của python 3.x sẽ tự động giải quyết, nếu sử dụng thư viện request, cần phải đặt cờ cho nó: allow_redirects=True.

ví dụ: r=requests.get('http://github.com',allow_redirects=True)

***khi cố gắng thu thập tất cả các liên kết ngoài, hãy tự hỏi***

- cố gắng thu thập dữ liệu gì?
=> phương hướng giải quyết là thử nghiệm chạy trên một vài trang web chỉ định trước

- trình thu thập của tôi có khám phá các trang web mới mà tôi không biết?

- liên kết đến một trang web mới hay là đi sâu vào trang web hiện tại?

- điều kiện để không truy cập vào các trang web không mong muốn cụ thể?

- có quan tâm đến ngôn ngữ của nội dung hay không?

- làm thế nào để bảo vệ mình trước hành động pháp lí nếu trình thu thập thông tin web của tôi thu hút sự chú ý của quản trị viên trên một trang web mà nó chạy qua?

In [5]:
from urllib.request import urlopen
from urllib.parse import urlparse
from urllib.error import HTTPError
from bs4 import BeautifulSoup

import re
import datetime
import random

In [11]:
pages = set()
random.seed(datetime.datetime.now())

# truy xuất danh sách tất cả các liên kết nội bộ được tìm thấy trên một trang
def getInternalLinks(bs, includeUrl):
    includeUrl = '{}://{}'.format(urlparse(includeUrl).scheme,
                                  urlparse(includeUrl).netloc)
    internalLinks = []
    # tìm tất cả links bắt đầu bằng '/'
    for link in bs.find_all('a',
                            href=re.compile('^(/|.*'+includeUrl+')')):
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in internalLinks:
                if (link.attrs['href'].startswith('/')):
                    internalLinks.append(includeUrl+link.attrs['href'])
                else:
                    internalLinks.append(link.attrs['href'])
    return internalLinks


# truy xuất danh sách tất cả các liên kết bên ngoài được tìm thấy trên một trang
def getExternalLinks(bs, excludeUrl):
    externalLinks = []
    # tìm tất cả các link bắt đầu với 'http' mà không chứa URL hiện tại
    for link in bs.find_all('a',
                            href=re.compile('^(http|www)((?!'+excludeUrl+').)*$')):
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in externalLinks:
                externalLinks.append(link.attrs['href'])
    return externalLinks


def getRandomExternalLink(startingPage):
    # trong trường hợp lỗi 403: Forbiden. hãy sử dụng người dùng giả mạo    
    html = urlopen(startingPage)
    bs = BeautifulSoup(html, 'html.parser')
    externalLinks = getExternalLinks(bs, 
                                     urlparse(startingPage).netloc)
    if len(externalLinks) == 0:
        print("No external links, looking around the site for one!")
        domain = '{}://{}'.format(urlparse(startingPage).scheme,
                                  urlparse(startingPage).netloc)
        internalLinks = getInternalLinks(bs, domain)
        return getRandomExternalLink(internalLinks[random.randint(0, len(internalLinks)-1)])
    else:
        return externalLinks[random.randint(0, len(externalLinks)-1)]
    

def followExternalonly(startingSite):
    externalLink = getRandomExternalLink(startingSite)
    print("Random external link is: {}".format(externalLink))
    try:
        followExternalonly(externalLink)
    except HTTPError as e:
        print(e)


followExternalonly('http://oreilly.com')

C:\Users\JOS UC\AppData\Local\Temp\ipykernel_23556\3373549060.py:2: DeprecationWarning: Seeding based on hashing is deprecated
since Python 3.9 and will be removed in a subsequent version. The only 
supported seed types are: None, int, float, str, bytes, and bytearray.
  random.seed(datetime.datetime.now())


Random external link is: https://www.linkedin.com/company/oreilly-media
HTTP Error 999: Request denied


***HTTPError: HTTP Error 999: Request denied***

In [12]:
#  lấy tất cả danh sách của URL bên ngoài được tìm thấy trên một site
allExtLinks = set()
allIntLinks = set()

def getAllExternalLinks(siteUrl):
    html = urlopen(siteUrl)
    domain = '{}://{}'.format(urlparse(siteUrl).scheme,
                              urlparse(siteUrl).netloc)
    bs = BeautifulSoup(html, 'html.parser')
    internalLinks = getInternalLinks(bs, domain)
    externalLinks = getExternalLinks(bs, domain)

    for link in externalLinks:
        if link not in allExtLinks:
            allExtLinks.add(link)
            print(link)
    for link in internalLinks:
        if link not in allIntLinks:
            getAllExternalLinks(link)


allIntLinks.add('http://oreily.com')
getAllExternalLinks('http://oreilly.com')


https://www.oreilly.com
https://www.oreilly.com/member/login/
https://www.oreilly.com/online-learning/try-now.html
https://www.oreilly.com/online-learning/teams.html
https://www.oreilly.com/online-learning/government.html
https://www.oreilly.com/online-learning/academic.html
https://www.oreilly.com/online-learning/individuals.html
https://www.oreilly.com/online-learning/features.html
https://www.oreilly.com/online-learning/courses.html
https://www.oreilly.com/online-learning/feature-certification.html
https://www.oreilly.com/online-learning/intro-interactive-learning.html
https://www.oreilly.com/online-learning/live-events.html
https://www.oreilly.com/online-learning/feature-answers.html
https://www.oreilly.com/online-learning/insights-dashboard.html
https://www.oreilly.com/radar/
https://www.oreilly.com/content-marketing-solutions.html
https://learning.oreilly.com/start-trial/
https://www.oreilly.com/about/oreilly-approach-to-generative-ai.html
https://www.oreilly.com/online-learning/

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x000001ECAF4AF9A0>>
Traceback (most recent call last):
  File "C:\Users\JOS UC\AppData\Roaming\Python\Python310\site-packages\ipykernel\ipkernel.py", line 788, in _clean_thread_parent_frames
    if phase != "start":
KeyboardInterrupt: 
